# **Big Data e Processamento Distribuído**
## **Exercício 2**
________________
### **_Luiz Flavio Pereira_**
___________________
### _Refazer os exercícios de MapReduce, desta vez com SparkRDD_
________________________

### Setup do programa

In [1]:
# Importacao das bibliotecas necessarias ao programa

import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='Tarefa2')

2021-12-16 23:12:34,156 WARN util.Utils: Your hostname, bigdatavm-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
2021-12-16 23:12:34,157 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2021-12-16 23:12:36,203 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Checagem da sessao do Spark

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

________________________
### Leitura do arquivo flights.csv

In [3]:
# Leitura do arquivo CSV

flights_file = 'file:///home/bigdata-vm/Desktop/BigDataAulas/Datasets/flights.csv'
flights_RDD = sc.textFile(flights_file)

________________
### **Questão a)** O tempo total de vôo de cada companhia

In [4]:
def total_voo_cia(line):
    record = line.split(',')
    return (record[1], float(record[9]))

total_voo = flights_RDD.map(total_voo_cia).take(500000)
total_voo_RDD = sc.parallelize(total_voo)

grouped_total_voo_RDD = total_voo_RDD.reduceByKey(lambda a,b: a+b).sortByKey()

print('Companhia  Tempo Total de Voo (minutos)')
print('---------------------------------------')

for key, values in grouped_total_voo_RDD.collect():
    print(f'{key}      {values}')

2021-12-16 23:12:43,538 WARN scheduler.TaskSetManager: Stage 2 contains a task of very large size (2211 KiB). The maximum recommended task size is 1000 KiB.


Companhia  Tempo Total de Voo (minutos)
---------------------------------------
19393      9959718.0
19690      449242.0
19790      7747256.0
19805      6273366.0
19930      2001295.0
19977      6757883.0
20304      3625141.0
20355      4194811.0
20366      4142864.0
20398      2391168.0
20409      3009096.0
20436      689589.0
20437      799733.0
21171      916098.0


________________
### **Questão b)** Qual é o destino mais visado

In [5]:
airports_visited = {}

def most_visited(line):
    record = line.split(',')
    return (record[4], int(1))

airports_visited_RDD = flights_RDD.map(most_visited)
total_airports_visited = airports_visited_RDD.reduceByKey(lambda a,b: a+b).take(500000)

for airports in total_airports_visited:
    airports_visited[airports[0]] = airports[1]

max_count = int(max(airports_visited.values()))
airport_most_visited = list(airports_visited.keys())[list(airports_visited.values()).index(max_count)]

print(f'O aeroporto mais visitado foi {airport_most_visited}, com {max_count} visitas.')

O aeroporto mais visitado foi ATL, com 30953 visitas.


________________
### **Questão c)** Quais aeroportos cada companhia passou

In [6]:
# Quais aeroportos cada companhia passou
def airports_origin(line):
    record = line.split(',')
    return (record[1], record[3])

airports = (flights_RDD.map(airports_origin).take(500000))
airports_RDD = sc.parallelize(airports)

grouped_airports_RDD = airports_RDD.groupByKey().sortByKey()

for key, values in grouped_airports_RDD.collect():
    print(key, set(values))
    print('\n')

2021-12-16 21:45:08,334 WARN scheduler.TaskSetManager: Stage 15 contains a task of very large size (1863 KiB). The maximum recommended task size is 1000 KiB.


19393 {'MSP', 'BHM', 'OKC', 'TUL', 'IND', 'AUS', 'ABQ', 'BUF', 'ATL', 'GRR', 'OMA', 'CHS', 'FNT', 'ROC', 'RIC', 'DAL', 'CLT', 'DTW', 'PDX', 'TUS', 'BDL', 'EYW', 'CAK', 'MCO', 'PHL', 'MHT', 'SAN', 'PNS', 'TPA', 'JAX', 'DEN', 'GEG', 'PBI', 'SLC', 'BKG', 'BOI', 'RSW', 'RDU', 'SAT', 'LAS', 'BNA', 'MEM', 'CLE', 'SMF', 'MDW', 'DSM', 'RNO', 'ICT', 'DCA', 'SJC', 'ALB', 'JAN', 'LIT', 'MSY', 'BOS', 'ISP', 'CMH', 'LBB', 'SFO', 'SNA', 'ECP', 'FLL', 'OAK', 'GSP', 'ONT', 'SEA', 'MAF', 'STL', 'AMA', 'PHX', 'LGA', 'DAY', 'HOU', 'MKE', 'LAX', 'ELP', 'BUR', 'CRP', 'EWR', 'PWM', 'MCI', 'SJU', 'HRL', 'PIT', 'IAD', 'SDF', 'PVD', 'ORF', 'BWI'}


19690 {'OGG', 'SMF', 'KOA', 'PPG', 'PDX', 'HNL', 'SFO', 'PHX', 'JFK', 'LAS', 'SJC', 'OAK', 'ITO', 'SAN', 'LAX', 'SEA', 'LIH'}


19790 {'CHA', 'MSP', 'BHM', 'OKC', 'TYS', 'MYR', 'TUL', 'GPT', 'IND', 'AUS', 'ABQ', 'BUF', 'ATL', 'GRR', 'OMA', 'CHS', 'MDT', 'TRI', 'FNT', 'ROC', 'RIC', 'MOB', 'CVG', 'CLT', 'OGG', 'DTW', 'PDX', 'TUS', 'PSP', 'JFK', 'ABE', 'BDL', 'IAH', 'E

________________
### **Questão d)** O vôo de maior distância de cada companhia

In [6]:
def max_flight(line):
    record = line.split(',')
    return (record[1], float(record[10]))

max_flight_by_cia = flights_RDD.map(max_flight).take(500000)
max_flight_by_cia_RDD = sc.parallelize(max_flight_by_cia)

grouped_max_flight_by_cia_RDD = max_flight_by_cia_RDD.groupByKey().sortByKey()

print('Companhia  Maior Voo (milhas)')
print('-----------------------------')

for key, values in grouped_max_flight_by_cia_RDD.collect():
    print(f'{key}      {max(set(values))}')

2021-12-16 23:15:47,182 WARN scheduler.TaskSetManager: Stage 15 contains a task of very large size (2211 KiB). The maximum recommended task size is 1000 KiB.


Companhia  Maior Voo (milhas)
-----------------------------
19393      2335.0
19690      4983.0
19790      4502.0
19805      3784.0
19930      2874.0
19977      4962.0
20304      1535.0
20355      2979.0
20366      1389.0
20398      1379.0
20409      2704.0
20436      1703.0
20437      2139.0
21171      2704.0


________________
### **Questão e)** Qual é o vôo mais frequente de cada companhia

In [7]:
dic_frequent_flights = {}
dic_frequent_flights_count = {}

def most_frequent(line):
    record = line.split(',')
    return (record[1], record[2])

most_frequent_flight_by_cia = flights_RDD.map(most_frequent).take(500000)
most_frequent_flight_by_cia_RDD = sc.parallelize(most_frequent_flight_by_cia)

grouped_most_frequent_flight_by_cia_RDD = most_frequent_flight_by_cia_RDD.groupByKey().sortByKey()

for i in grouped_most_frequent_flight_by_cia_RDD.collect():
    dic_frequent_flights[i[0]] = list(i[1])
    
for cia in dic_frequent_flights.keys():
    for id_flight in dic_frequent_flights[cia]:
        if id_flight in dic_frequent_flights_count:
            dic_frequent_flights_count[id_flight] += 1
        else:
            dic_frequent_flights_count[id_flight] = 1
            
    max_count = int(max(dic_frequent_flights_count.values()))
    id_flight_max = list(dic_frequent_flights_count.keys())[list(dic_frequent_flights_count.values()).index(max_count)]
    dic_frequent_flights_count.clear()
    
    print(f'O voo mais frequente da companhia {cia} foi o {id_flight_max} com {max_count} voos.')

2021-12-16 23:15:58,028 WARN scheduler.TaskSetManager: Stage 24 contains a task of very large size (1937 KiB). The maximum recommended task size is 1000 KiB.


O voo mais frequente da companhia 19393 foi o 583 com 170 voos.
O voo mais frequente da companhia 19690 foi o 1 com 30 voos.
O voo mais frequente da companhia 19790 foi o 1258 com 60 voos.
O voo mais frequente da companhia 19805 foi o 29 com 60 voos.
O voo mais frequente da companhia 19930 foi o 64 com 150 voos.
O voo mais frequente da companhia 19977 foi o 1246 com 75 voos.
O voo mais frequente da companhia 20304 foi o 5325 com 90 voos.
O voo mais frequente da companhia 20355 foi o 409 com 120 voos.
O voo mais frequente da companhia 20366 foi o 2514 com 60 voos.
O voo mais frequente da companhia 20398 foi o 2726 com 60 voos.
O voo mais frequente da companhia 20409 foi o 2054 com 59 voos.
O voo mais frequente da companhia 20436 foi o 143 com 50 voos.
O voo mais frequente da companhia 20437 foi o 221 com 46 voos.
O voo mais frequente da companhia 21171 foi o 251 com 60 voos.
